# BM25 Retrieval with Query Rewriting with SDM in PyTerrier

This Jupyter notebook implements query rewriting with SDM for BM25 retrieval.
The notebook itself is a bit more condensed.
For a more detailed notebook, please look at [pyterrier-bm25.ipynb](pyterrier-bm25.ipynb).

### Step 1: Import everything and load variables

In [2]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./iranthology-dataset-tira')


I will use a small hardcoded example located in ./iranthology-dataset-tira.
The output directory is /tmp/


### Step 2: Load the Data

In [3]:
print('Step 2: Load the data.')

queries = pt.io.read_topics(input_directory + '/queries.xml', format='trecxml')

documents = [json.loads(i) for i in open(input_directory + '/documents.jsonl', 'r')]


Step 2: Load the data.


### Step 3: Create the Index

In [4]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100}, blocks=True)
index_ref = iter_indexer.index(tqdm(documents))

Step 3: Create the Index.


 30%|██████████████████████████████▋                                                                      | 16327/53673 [00:05<00:08, 4309.88it/s]

06:04:16.975 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 53673/53673 [00:13<00:00, 4122.63it/s]


06:04:26.071 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


### Step 4: Create Retrieval Pipeline

In [5]:
sdm = pt.rewrite.SDM()
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", verbose=True)

retrieval_pipeline = sdm >> bm25

### Step 5: Create the run

In [6]:
print('Step 5: Create Run.')

run = retrieval_pipeline(queries)

Step 5: Create Run.


BR(BM25): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  7.01q/s]


In [7]:
print('We look at the first 10 results of the run (query has ben rewritten):\n')
run.head(10)

We look at the first 10 results of the run (query has ben rewritten):



,qid,docid,docno,rank,score,query,query_0
0,1,49659,2021.ipm_journal-ir0anthology0volumeA58A1.6,0,16.808093,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
1,1,27490,2011.spire_conference-2011.10,1,15.805688,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
2,1,19930,2019.cikm_conference-2019.346,2,15.560707,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
3,1,39429,2021.tist_journal-ir0anthology0volumeA12A2.4,3,15.190721,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
4,1,33009,2013.wwwconf_conference-2013c.302,4,14.881860,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
5,1,33172,2018.wwwconf_conference-2018.13,5,14.758040,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
6,1,23061,2010.cikm_conference-2010.284,6,14.733129,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
7,1,28878,2012.wwwconf_conference-2012c.37,7,14.696894,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
8,1,14388,2016.fire_conference-2016w.51,8,14.657960,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries
9,1,13049,2020.clef_conference-2020w.101,9,14.575418,detect health queries #combine:0=0.1:wmodel=or...,detect health related queries


### Step 6: Persist Run

In [8]:
print('Step 6: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='BM25-SDM', depth=1000)

print('Done :)')

Step 6: Persist Run.
Done :)
